In [1]:
%pip install torch
%pip install transformers
%pip install json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 22.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 46.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 189.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 168.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 22.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 66.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 62.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 134.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 69.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 75.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import json
import pandas as pd

In [3]:
# Load FinBERT Model and Tokenizer
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [4]:
#importing data
df_news = pd.read_excel(r'news_headlines.xlsx')
df_news.head()

,Date,Headline,Site_Name
0,2024-04-01,Adani Green Energy Operationalizes Multiple So...,Energetica India Magazine
1,2024-04-01,"Adani Group Stocks In Heavy Demand, Adani Ener...",NDTV
2,2024-04-01,Adani Group is back to its rapid expansion move,India Shipping News
3,2024-04-01,Adani Total Gas kicks off production at Barsan...,Sarkaritel.com
4,2022-08-01,"Asia’s two richest men Ambani, Adani start to ...",Al Jazeera


In [5]:
# Extract and print article text
for index, entry in df_news.iterrows():
    article_text = entry['Headline']
    
    # print(article_text)
    # Tokenize the text
    inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)
    
    # Run the model to get logits
    outputs = model(**inputs)
    logits = outputs.logits
    
    # Apply Softmax to get sentiment probabilities
    sentiment_probs = torch.nn.functional.softmax(logits, dim=-1)
    
    # Get the predicted sentiment label and its probability
    sentiment_label = torch.argmax(sentiment_probs).item()
    confidence_score = sentiment_probs[0][sentiment_label].item()
    
    # FinBERT Sentiment Labels
    sentiment_classes = ["Positive", "Negative", "Neutral"]
    print(f"Predicted Sentiment: {sentiment_classes[sentiment_label]} (Score: {confidence_score:.2f})")
    df_news.loc[index, 'sentiment_label'] = sentiment_classes[sentiment_label]
    df_news.loc[index, 'confidence_score'] = confidence_score
    
 

Predicted Sentiment: Neutral (Score: 0.79)
Predicted Sentiment: Positive (Score: 0.91)
Predicted Sentiment: Positive (Score: 0.80)
Predicted Sentiment: Neutral (Score: 0.87)
Predicted Sentiment: Neutral (Score: 0.77)
Predicted Sentiment: Negative (Score: 0.97)
Predicted Sentiment: Positive (Score: 0.93)
Predicted Sentiment: Positive (Score: 0.92)
Predicted Sentiment: Neutral (Score: 0.95)
Predicted Sentiment: Positive (Score: 0.51)
Predicted Sentiment: Neutral (Score: 0.92)
Predicted Sentiment: Neutral (Score: 0.81)
Predicted Sentiment: Neutral (Score: 0.94)
Predicted Sentiment: Neutral (Score: 0.86)
Predicted Sentiment: Neutral (Score: 0.57)
Predicted Sentiment: Neutral (Score: 0.63)
Predicted Sentiment: Neutral (Score: 0.58)
Predicted Sentiment: Negative (Score: 0.50)
Predicted Sentiment: Positive (Score: 0.67)
Predicted Sentiment: Neutral (Score: 0.66)
Predicted Sentiment: Positive (Score: 0.86)
Predicted Sentiment: Neutral (Score: 0.82)
Predicted Sentiment: Neutral (Score: 0.65)
Pr

In [6]:
df_news.head()

,Date,Headline,Site_Name,sentiment_label,confidence_score
0,2024-04-01,Adani Green Energy Operationalizes Multiple So...,Energetica India Magazine,Neutral,0.787007
1,2024-04-01,"Adani Group Stocks In Heavy Demand, Adani Ener...",NDTV,Positive,0.911970
2,2024-04-01,Adani Group is back to its rapid expansion move,India Shipping News,Positive,0.802912
3,2024-04-01,Adani Total Gas kicks off production at Barsan...,Sarkaritel.com,Neutral,0.869320
4,2022-08-01,"Asia’s two richest men Ambani, Adani start to ...",Al Jazeera,Neutral,0.766120


In [7]:
df_news.to_excel('FinBERT_Results.xlsx', index= False)